# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [106]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [107]:
# Hacé la separación en esta celda

X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
y.astype('float')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(5100, 58) (5100,)
(1276, 58) (1276,)


In [108]:
# Esto es lo que hace
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [109]:
# Creá en esta celda la variable param_grid

param_grid = [
    {'max_depth': [1,2,3,4,5], 'max_features': [1,2,3,4,5]},
]

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [110]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)


A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [111]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5],
                          'max_features': [1, 2, 3, 4, 5]}],
             pre_dispatch='2*n_jobs', 

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [112]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [113]:
# Mostrá los grid_scores en esta celda
grid_scores = grid_search.cv_results_['mean_train_score']
grid_scores

array([-9.55588857e+08, -9.57468890e+08, -9.72983861e+08, -9.23628746e+08,
       -9.74037208e+08, -9.27697741e+08, -9.22782382e+08, -9.18594195e+08,
       -9.17251310e+08, -9.26891537e+08, -9.46259883e+08, -9.06038074e+08,
       -8.53680623e+08, -7.71936220e+08, -8.64641322e+08, -9.60984110e+08,
       -9.35611617e+08, -7.91252565e+08, -6.90511634e+08, -8.74058455e+08,
       -8.39258972e+08, -8.16057480e+08, -7.53021816e+08, -8.00184436e+08,
       -8.24706576e+08])

In [115]:
results = pd.DataFrame(grid_search.cv_results_, columns=['rank_test_score','params','mean_test_score','std_test_score','mean_train_score','std_train_score'])
results.sort_values(by=['rank_test_score'],inplace=True)
results.head()

rank_test_score                               params  mean_test_score  \
18                1  {'max_depth': 4, 'max_features': 4}   -713698124.971   
13                2  {'max_depth': 3, 'max_features': 4}   -778662848.865   
22                3  {'max_depth': 5, 'max_features': 3}   -783031651.418   
17                4  {'max_depth': 4, 'max_features': 3}   -798821216.107   
23                5  {'max_depth': 5, 'max_features': 4}   -820222490.251   

    std_test_score  mean_train_score  std_train_score  
18    10026636.537    -690511634.402      9351525.166  
13    83155381.455    -771936219.722     94274929.912  
22   107876097.685    -753021815.939    106376140.074  
17    86551142.368    -791252564.660     83155545.746  
23   130461467.752    -800184435.788    127982874.903

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [116]:
# Mostrás los resultados en esta celda
grid_search.best_estimator_
grid_search.best_params_

{'max_depth': 4, 'max_features': 4}

Convertimos a RMSE.

In [117]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [118]:
param_grid2 = [
    {
    "min_samples_split": [2, 10, 20], "max_depth": [None, 2, 5, 10, 15], "min_samples_leaf": [1, 5, 10, 15], "max_leaf_nodes": [None, 5, 10, 20]
},
]
grid_search = GridSearchCV(tree_reg, param_grid2, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [None, 2, 5, 10, 15],
                          'max_leaf_nodes': [None, 5, 10, 20],
                          'min_

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [119]:
optimised_decision_tree = grid_search.best_estimator_
optimised_decision_tree

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=15, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [120]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [121]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [122]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [123]:
predicciones.head(10)

Valor real  Valor Pred  Dif(+/-)
0   80000.000  103438.660 23438.660
1  128000.000  135705.882  7705.882
2  150000.000  156075.759  6075.759
3   85000.000  102400.991 17400.991
4  135000.000  135571.622   571.622
5  135000.000  109560.000 25440.000
6   68000.000   75181.250  7181.250
7  110000.000  140444.444 30444.444
8  134000.000  158431.250 24431.250
9  110000.000   76701.202 33298.798